In [1]:
!pip install transformers==3

In [2]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import pandas as pd
import re

In [4]:
from google.colab import files

In [60]:
uploaded = files.upload()

Saving qa.xlsx to qa.xlsx


In [61]:
import io 
reponses = pd.read_excel(io.BytesIO(uploaded['qa.xlsx']))

In [53]:
paras = []
para = ""
for a in reponses.answer:
    if (len(para.split(" ")) + len(a.split(" "))) > 400:
        paras.append(para)
        para = ""
        para += " " + a
    
    else:
        para += " " + a
paras.append(para)

Load the BertForQuestionAnswering model and the tokenizer.

In [9]:
#Model
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [14]:
def answerme(question):
    answers = []
    for para in paras:
        encoding = tokenizer.encode_plus(text=question,text_pair=para)

        inputs = encoding['input_ids']  #Token embeddings
        sentence_embedding = encoding['token_type_ids']  #Segment embeddings
        tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens

        start_scores, end_scores = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))

        start_index = torch.argmax(start_scores)

        end_index = torch.argmax(end_scores)

        answer = ' '.join(tokens[start_index:end_index+1])
        corrected_answer = ''

        for word in answer.split():

            #If it's a subword token
            if word[0:2] == '##':
                corrected_answer += word[2:]
            else:
                corrected_answer += ' ' + word
        answers.append(corrected_answer)

    for i in range (len(answers)-1, -1, -1):
      a = re.sub('[^A-Za-z0-9]+', '', answers[i])
      if answers[i] == ' [CLS]':
        answers.pop(i)
    if len(answers) == 0:
      answers.append("Sorry I don't understand the question")
    return answers[0]

In [62]:
uploaded = files.upload()

Saving qa.csv to qa.csv


In [63]:
questions = pd.read_csv(io.BytesIO(uploaded['qa.csv']))

In [59]:
print("type 'exit' or 'e' to end the discussion")
print("type 'question' or 'q' to ask standard question")
while True:
  q = str(input("Question : ")).strip()
  if q == "exit" or q == "e":
    break
  if q =="question" or q == "q":
    q = questions.sample(1).question.values[0]
    print(q)
  a = answerme(q)
  print("Answer :", a)
  

type 'exit' or 'e' to end the discussion
type 'question' or 'q' to ask standard question
Question : q
What is Corpus ?
Answer :  a collection of text documents
Question : q
What is Fasttext ?
Answer :  a library for learning of word embeddings and text classification
Question : what is nlp
Answer :  natural language processing
Question : what is tokenization
Answer :  the method of dividing the text into various tokens
Question : exit
